In [1]:
import pandas as pd
import numpy as np
import os
import json

In [8]:
model_path = '../Modelo/MiBiciTuBici'
model_output_path = f'{model_path}/model-output'
processed_output_path = f'{model_path}/processed-model-output/'
experiment_1_path = f'{model_output_path}/experiment-1'
experiment_2_path = f'{model_output_path}/experiment-2'

# --> Sanity checks
if not os.path.exists(model_output_path) or len(os.listdir(model_output_path)) == 0:
    raise Exception('No model output is available. Run the AnyLogic experiment and try again.')
if not os.path.exists(experiment_1_path) or len(os.listdir(experiment_1_path)) == 0:
    raise Exception('No model output is available for experiment 1. Run the AnyLogic experiment and try again.')
if not os.path.exists(experiment_2_path) or len(os.listdir(experiment_2_path)) == 0:
    raise Exception('No model output is available for experiment 2. Run the AnyLogic experiment and try again.')

if not os.path.exists(processed_output_path):
    os.mkdir(processed_output_path)

experiment_1_files = os.listdir(experiment_1_path)
experiment_2_files = os.listdir(experiment_2_path)

for file in experiment_1_files:
    if not file.startswith('output') and not file.endswith('.json'):
        raise Exception(f'File {file} is not recognized. Aborting.')

for file in experiment_2_files:
    if not file.startswith('output') and not file.endswith('.json'):
        raise Exception(f'File {file} is not recognized. Aborting.')

n_replications = len(experiment_1_files)

statistics = (
    'co2Avoided',
)

results_exp_1 = { f'{statistic}': [] for statistic in statistics }
results_exp_2 = { f'{statistic}': [] for statistic in statistics }
results_difference = { f'{statistic}': [] for statistic in statistics }

In [31]:
for output_file in experiment_1_files:
    with open(f'{model_output_path}/experiment-1/{output_file}', 'r', encoding='utf8') as f:
        data = json.loads(f.read())

    for statistic in statistics:
        results_exp_1[statistic].append(data[statistic])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [36]:
with open(f'{model_output_path}/experiment-1/output-rep1013.json', 'r') as f:
    data = json.load(f)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 192: invalid continuation byte

In [4]:
for output_file in experiment_2_files:
    with open(f'{model_output_path}/experiment-2/{output_file}') as f:
        data = json.load(f)

    stations_population_data_exp_2.append(data['estaciones'])

    for statistic in statistics:
        results_exp_2[statistic].append(data[statistic])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 185: invalid continuation byte

In [18]:
hasta = 3

s = np.std(results_exp_2['co2Avoided'][:hasta])
n = len(results_exp_2['co2Avoided'][:hasta])

print(s/np.sqrt(n))
print(1.96*s)

if s/np.sqrt(n) <= 1.96*s: 
    print('suficiente')

9839.43526856974
33403.1075337779
suficiente
